## Calculate ERPs

In [1]:
#Importing files and modules

import numpy as np
import matplotlib.pyplot as plt
from os import path
#%matplotlib notebook
from brainpipe.system import study
from brainpipe.visual import *
from brainpipe.statistics import *
from mne.baseline import rescale
from mne.filter import filter_data
from mne.stats import *

## User variables

In [2]:
low_pass_filter = 10.
sf = 512.
norm_mode = 'mean' #'ratio' 'mean' 'percent' 
baseline = [896 , 1024]
data_to_use = [896, 1536]

# Compute ERPs and Stats

In [ ]:
st = study('Olfacto')
path_data = path.join (st.path, 'database/TS_E_all_by_odor_th40_art400_30_250_5s_concatOK/')
elec = 84

#Load files
name_early = 'PIRJ_concat_odor_bad_bipo.npz'
name_late = 'PIRJ_concat_odor_good_bipo.npz'
data_early = np.load(path.join(path_data, name_early))
data_late = np.load(path.join(path_data, name_late))
data_early, channel, label, data_late = data_early['x'], data_early['channel'], data_early['label'], data_late['x']

# Select data for one elec + name :
data_elec_early = data_early[elec,:,:]
data_elec_late = data_late[elec,:,:]
ntrials = len(data_elec_early[2])
print ('Channel : ', channel[elec], 'Label : ', label[elec], 'N_trials :', ntrials, 
       'One elec shape : ', data_elec_early.shape)

#Filter data for one elec (all trials):
data_elec_early = np.array(data_elec_early, dtype='float64')
data_elec_late = np.array(data_elec_late, dtype='float64')
data_early_to_filter = np.swapaxes(data_elec_early, 0, 1)
data_late_to_filter = np.swapaxes(data_elec_late, 0, 1)
filtered_data_early = filter_data(data_early_to_filter, sfreq=512, l_freq=None, h_freq=low_pass_filter, method='fir', phase='zero-double')
filtered_data_late = filter_data(data_late_to_filter, sfreq=512, l_freq=None, h_freq=low_pass_filter, method='fir', phase='zero-double')
print ('Size of filtered data early :', filtered_data_early.shape, 'filtered data late : ', filtered_data_late.shape,)

#Normalize the non-averaged data (all trials)
times = np.arange(filtered_data_early.shape[1])
print ('time points : ', times.shape)
norm_filtered_data_early = rescale(filtered_data_early, times=times, baseline=baseline, mode=norm_mode)
norm_filtered_data_late = rescale(filtered_data_late, times=times, baseline=baseline, mode=norm_mode)
print ('Size norm & filtered data 0 : ', norm_filtered_data_early.shape, norm_filtered_data_late.shape,)

# =======================================  STATISTICS  =====================================
# Range of the data to compute
data_range = range(data_to_use[0], data_to_use[1])

# Get all three learning files (Filtered data)
data_learn_0 = norm_filtered_data_early[:, data_range]
data_learn_1 = norm_filtered_data_late[:, data_range,]
print ('-> Shape of the selected data for learn 0', data_learn_0.shape, 'learn 1', data_learn_1.shape,)

#reshape data to have the exact same nb od trials (mandatory for t-tests)
data_learn_0_rand = data_learn_0[np.random.randint(data_learn_0.shape[0], size=data_learn_1.shape[0]), :] #reshape early_data to fit late_data shape
print ('rand early matrix', data_learn_0_rand.shape)
X = data_learn_0_rand - data_learn_1 #the last dimension needs to be time
T0, p_values, H0 = permutation_t_test(X, n_permutations=1000, tail=0, n_jobs=1, verbose=None)

# =======================PREPARE DATA TO PLOT AND PLOT THE ERPs=====================================
# Data to plot :
data_learn_to_plot = np.concatenate([data_learn_0_rand, data_learn_1,], axis=0)
data_learn_to_plot = data_learn_to_plot.swapaxes(0,1)
print('-> Shape of data to plot : ', data_learn_to_plot.shape)

# Time vector & label vector:
times_plot = 1000 * np.arange((baseline[0] - baseline[1]), data_learn_to_plot.shape[0]-baseline[1] + baseline[0],) / sf
print('-> Shape of time vector : ', times_plot.shape)
label_learn_0 = np.zeros(data_learn_0_rand.shape[0], dtype='int64')
label_learn_1 = np.ones(data_learn_1.shape[0], dtype='int64')
print('label size and values : 0 : ', label_learn_0.shape, '1 : ', label_learn_1.shape,)
y = np.concatenate([label_learn_0, label_learn_1,])
print('-> the y label', y.shape)

#Prepare the plot
fig = plt.figure(0, figsize=(12, 7))
ax = fig.add_subplot(111)
fig.subplots_adjust(top=0.85)
ax.set_xlabel('Times (ms)', fontsize=12)
ax.set_ylabel('Potential', fontsize=12)

#Plot the Data
BorderPlot(times_plot, data_learn_to_plot, y=y, kind='sem', alpha=0.2, color=['m','b'], linewidth=2, ncol=1, legend= ['Early Learning', 'Late Learning'],
          title='PIRJ_ERP_Odor_bipo_'+norm_mode+'_'+str(channel[elec])+'_'+str(label[elec])+' elec_num: '+str(elec)+'_ntrials:'+str(ntrials),)
plt.gca()
lines = [0] #time vector is in ms
addPval(plt.gca(), p_values, p=0.05, x=times_plot, y=0.5, color='m', lw=3)
addLines(plt.gca(), vLines=lines, vColor=['firebrick'], vWidth=[2], hLines=[0], hColor=['#000000'], hWidth=[2])
plt.legend(fontsize='small')
plt.grid()
plt.show()         


## Compute ERPs for odor groups

In [ ]:
st = study('Olfacto')
path_data = path.join (st.path, 'database/TS_E_all_by_odor_th40_art400_30_250_5s_concatOK/')
elec = 26

#Load files
badname = 'PIRJ_concat_odor_bad_bipo.npz'
goodname = 'PIRJ_concat_odor_good_bipo.npz'
data_bad = np.load(path.join(path_data, badname))
data_good = np.load(path.join(path_data, goodname))
data_bad, channel, label, data_good = data_bad['x'], data_bad['channel'], data_bad['label'], data_good['x']
print (data_bad.shape, data_good.shape)

# Select data for one elec + name :
data_elec_bad = data_bad[elec,:,:]
data_elec_good = data_good[elec,:,:]
ntrials = len(data_elec_bad[2])
print ('Channel : ', channel[elec], 'Label : ', label[elec], 'N_trials :', ntrials, 
       'Bad shape : ', data_elec_bad.shape, 'Good shape : ', data_elec_good.shape)

#Filter data for one elec (all trials):
data_elec_bad = np.array(data_elec_bad, dtype='float64')
data_elec_good = np.array(data_elec_good, dtype='float64')
data_bad_to_filter = np.swapaxes(data_elec_bad, 0, 1)
data_good_to_filter = np.swapaxes(data_elec_good, 0, 1)
filtered_data_bad = filter_data(data_bad_to_filter, sfreq=512, l_freq=None, h_freq=low_pass_filter, method='fir', phase='zero-double')
filtered_data_good = filter_data(data_good_to_filter, sfreq=512, l_freq=None, h_freq=low_pass_filter, method='fir', phase='zero-double')
print ('Size of filtered data bad :', filtered_data_bad.shape, 'filtered data good : ', filtered_data_good.shape,)

#Normalize the non-averaged data (all trials)
times = np.arange(filtered_data_bad.shape[1])
print ('time points : ', times.shape)
norm_filtered_data_bad = rescale(filtered_data_bad, times=times, baseline=baseline, mode=norm_mode)
norm_filtered_data_good = rescale(filtered_data_good, times=times, baseline=baseline, mode=norm_mode)
print ('Size norm & filtered data 0 : ', norm_filtered_data_bad.shape, norm_filtered_data_good.shape,)

# =======================================  STATISTICS  =====================================
# Range of the data to compute
data_range = range(data_to_use[0], data_to_use[1])

# Get all three learning files (Filtered data)
data_bad = norm_filtered_data_bad[:, data_range]
data_good = norm_filtered_data_good[:, data_range,]
print ('-> Shape of the selected data for learn 0', data_bad.shape, 'learn 1', data_good.shape,)

# #reshape data to have the exact same nb od trials (mandatory for t-tests)
# if data_bad.shape[0] > data_good.shape[0]:
#     data_bad_stats = data_bad[np.random.randint(data_bad.shape[0], size=data_good.shape[0]), :] #reshape bad_data to fit good_data shape
#     data_good_stats = data_good
#     print ('rand bad matrix', data_bad.shape)
# if data_bad.shape[0] < data_good.shape[0]:
#     data_good_rand = data_good[np.random.randint(data_good.shape[0], size=data_bad.shape[0]), :]
#     data_bad_stats = data_bad
#     print ('rand good matrix', data_good.shape)
# X = data_bad_stats - data_good_stats #the last dimension needs to be time
# T0, p_values, H0 = permutation_t_test(X, n_permutations=1000, tail=0, n_jobs=1, verbose=None)

# =======================PREPARE DATA TO PLOT AND PLOT THE ERPs=====================================
# Data to plot :
data_learn_to_plot = np.concatenate([data_bad, data_good,], axis=0)
data_learn_to_plot = data_learn_to_plot.swapaxes(0,1)
print('-> Shape of data to plot : ', data_learn_to_plot.shape)

# Time vector & label vector:
times_plot = 1000 * np.arange((baseline[0] - baseline[1]), data_learn_to_plot.shape[0]-baseline[1] + baseline[0],) / sf
print('-> Shape of time vector : ', times_plot.shape)
label_bad = np.zeros(data_bad.shape[0], dtype='int64')
label_good = np.ones(data_good.shape[0], dtype='int64')
print('label size and values : 0 : ', label_bad.shape, '1 : ', label_good.shape,)
y = np.concatenate([label_bad, label_good,])
print('-> the y label', y.shape)

#Prepare the plot
fig = plt.figure(0, figsize=(12, 7))
ax = fig.add_subplot(111)
fig.subplots_adjust(top=0.85)
ax.set_xlabel('Times (ms)', fontsize=12)
ax.set_ylabel('Potential', fontsize=12)

#Plot the Data
BorderPlot(times_plot, data_learn_to_plot, y=y, kind='sem', alpha=0.2, color=['m','b'], linewidth=2, ncol=1, legend= ['bad group', 'good group'],
          title='PIRJ_ERP_Odor_bipo_'+norm_mode+'_'+str(channel[elec])+'_'+str(label[elec])+' elec_num: '+str(elec)+'_ntrials:'+str(ntrials),)
plt.gca()
lines = [0] #time vector is in ms
addPval(plt.gca(), p_values, p=0.05, x=times_plot, y=0.5, color='m', lw=3)
addLines(plt.gca(), vLines=lines, vColor=['firebrick'], vWidth=[2], hLines=[0], hColor=['#000000'], hWidth=[2])
plt.legend(fontsize='small')
plt.grid()
plt.show()         


# Plot all ERPs with stats for learning cond

In [ ]:
st = study('Olfacto')
path_data = path.join (st.path, 'database/TS_E_all_cond_by_block_trigs_th40_art400_30_250_5s_learning2blocks/')
subjects = ['SEMC','PIRJ','LEFC','MICP','CHAF','VACJ',] 
nelec=10

for su in subjects:
    for elec in range(nelec):
        name_early = su+'_E1E2_concat_early_bipo.npz'
        data_early = np.load(path.join(path_data, name_early))
        data_early = data_early['channel'][elec]
        print (su, data_early)


In [ ]:
st = study('Olfacto')
path_data = path.join (st.path, 'database/TS_E_all_cond_by_block_trigs_th40_art400_30_250_5s_learning2blocks/')

subjects = ['SEMC','PIRJ','LEFC','MICP','CHAF','VACJ',] 

n_elec = {
    'CHAF' : 107,
    'VACJ' : 139, 
    'SEMC' : 107,
    'PIRJ' : 106,
    'LEFC' : 193,
    'MICP' : 105,
}


for su in subjects:
    for elec in range(0, n_elec[su],1):
        #Load files
        name_early = su+'_E1E2_concat_early_bipo.npz'
        name_late = su+'_E1E2_concat_late_bipo.npz'
        data_early = np.load(path.join(path_data, name_early))
        data_late = np.load(path.join(path_data, name_late))
        data_early, channel, label, data_late = data_early['x'], data_early['channel'], data_early['label'], data_late['x']

        # Select data for one elec + name :
        data_elec_early = data_early[elec,:,:]
        data_elec_late = data_late[elec,:,:]
        ntrials = len(data_elec_early[2])
        print ('Channel : ', channel[elec], 'Label : ', label[elec], 'N_trials :', ntrials, 'One elec shape : ', data_elec_early.shape)

        #Filter data for one elec (all trials):
        data_elec_early = np.array(data_elec_early, dtype='float64')
        data_elec_late = np.array(data_elec_late, dtype='float64')
        data_early_to_filter = np.swapaxes(data_elec_early, 0, 1)
        data_late_to_filter = np.swapaxes(data_elec_late, 0, 1)
        filtered_data_early = filter_data(data_early_to_filter, sfreq=512, l_freq=None, h_freq=low_pass_filter, method='fir', phase='zero-double')
        filtered_data_late = filter_data(data_late_to_filter, sfreq=512, l_freq=None, h_freq=low_pass_filter, method='fir', phase='zero-double')
        print ('Size of filtered data early :', filtered_data_early.shape, 'filtered data late : ', filtered_data_late.shape,)     

        #Normalize the non-averaged data (all trials)
        times = np.arange(filtered_data_early.shape[1])
        print ('time points : ', times.shape)
        norm_filtered_data_early = rescale(filtered_data_early, times=times, baseline=baseline, mode=norm_mode)
        norm_filtered_data_late = rescale(filtered_data_late, times=times, baseline=baseline, mode=norm_mode)
        print ('Size norm & filtered data 0 : ', norm_filtered_data_early.shape, norm_filtered_data_late.shape,)

        # =======================================  STATISTICS  =====================================
        # Range of the data to compute
        data_range = range(data_to_use[0], data_to_use[1])

        # Get all three learning files (Filtered data)
        data_learn_0 = norm_filtered_data_early[:, data_range]
        data_learn_1 = norm_filtered_data_late[:, data_range,]
        print ('-> Shape of the selected data for learn 0', data_learn_0.shape, 'learn 1', data_learn_1.shape,)

        #reshape data to have the exact same nb od trials (mandatory for t-tests)
        data_learn_0_rand = data_learn_0[np.random.randint(data_learn_0.shape[0], size=data_learn_1.shape[0]), :] #reshape early_data to fit late_data shape
        print ('rand early matrix', data_learn_0_rand.shape)
        X = data_learn_0_rand - data_learn_1 #the last dimension needs to be time
        T0, p_values, H0 = permutation_t_test(X, n_permutations=1000, tail=0, n_jobs=1, verbose=None)

        # =======================PREPARE DATA TO PLOT AND PLOT THE ERPs=====================================            
        #if p_values.min() <= 0.05:
        # Data to plot :
        data_learn_to_plot = np.concatenate([data_learn_0_rand, data_learn_1,], axis=0)
        data_learn_to_plot = data_learn_to_plot.swapaxes(0,1)
        print('-> Shape of data to plot : ', data_learn_to_plot.shape)

        # Time vector & label vector:
        times_plot = 1000 * np.arange((baseline[0] - baseline[1]), data_learn_to_plot.shape[0]-baseline[1] + baseline[0],) / sf
        print('-> Shape of time vector : ', times_plot.shape)
        label_learn_0 = np.zeros(data_learn_0_rand.shape[0], dtype='int64')
        label_learn_1 = np.ones(data_learn_1.shape[0], dtype='int64')
        print('label size and values : 0 : ', label_learn_0.shape, '1 : ', label_learn_1.shape,)
        y = np.concatenate([label_learn_0, label_learn_1,])
        print('-> the y label', y.shape)

        #Prepare the plot
        fig = plt.figure(0, figsize=(12, 7))
        ax = fig.add_subplot(111)
        fig.subplots_adjust(top=0.85)
        ax.set_xlabel('Times (ms)', fontsize=12)
        ax.set_ylabel('Potential', fontsize=12)

        #Plot the Data
        BorderPlot(times_plot, data_learn_to_plot, y=y, kind='sem', alpha=0.2, color=['m', 'b'], linewidth=2, ncol=1, legend= ['Early Learning', 'Late Learning'],
                  title=su+'_ERP_Odor_bipo_'+norm_mode+'_'+str(channel[elec])+'_'+str(label[elec])+' elec_num: '+str(elec)+'_ntrials:'+str(ntrials),)
        plt.gca()
        lines = [0] #time vector is in ms
        addPval(plt.gca(), p_values, p=0.05, x=times_plot, y=2, color='c', lw=3)
        addLines(plt.gca(), vLines=lines, vColor=['firebrick'], vWidth=[2], hLines=[0], hColor=['#000000'], hWidth=[2])
        plt.legend(fontsize='small')
        plt.grid()
        #plt.show()         

# =========================  SAVE PLOTS of ERPs   =================================================
        rep = path.join(st.path, 'feature/ERP_Encoding_all_bipo_250ms_mean_thr40_art400_30_250_learning_2blocks/',su)
        fname = (rep + '_E1E2_ERP_concat_all_bipo_' + channel [elec] +'_'+str(elec)+'_'+label[elec]+'.png')
        print (fname)
        plt.savefig(fname, dpi=300, bbox_inches='tight')
        plt.clf()
        plt.close()
        del channel, ntrials, label, data_learn_0_rand, data_learn_0, data_learn_1,

## Plot ERPs for Odor groups

In [3]:
st = study('Olfacto')
path_data = path.join (st.path, 'database/TS_E_all_by_odor_th40_art400_30_250_5s_concatOK/')
save_path = path.join(st.path, 'feature/ERP_Groups_Odors_250ms_rescale_filtered_stats/Significant/')

test = False

if test == True:
    n_elec = {'SEMC' :2}
    subjects = ['SEMC']
else :
    subjects = ['SEMC','PIRJ','LEFC','MICP','CHAF','VACJ'] 
    n_elec = {
    'CHAF' : 107,
    'VACJ' : 139, 
    'SEMC' : 107,
    'PIRJ' : 106,
    'LEFC' : 193,
    'MICP' : 105,
        }

for su in subjects:
    all_elec_p_val = np.array([])
    all_elec_T =np.array([])
    for elec in range(0, n_elec[su],1):
        #Load files
        badname = su+'_concat_odor_bad_bipo.npz'
        goodname = su+'_concat_odor_good_bipo.npz'
        data_bad = np.load(path.join(path_data, badname))
        data_good = np.load(path.join(path_data, goodname))
        data_bad, channel, label, data_good = data_bad['x'], data_bad['channel'], data_bad['label'], data_good['x']

        # Select data for one elec + name :
        data_elec_bad = data_bad[elec,:,:]
        data_elec_good = data_good[elec,:,:]
        ntrials = str(data_elec_bad.shape[1])+'/'+ str(data_elec_good.shape[1])
        print ('Channel : ', channel[elec], 'Label : ', label[elec], 'N_trials :', ntrials, 
               'Bad shape : ', data_elec_bad.shape, 'Good shape : ', data_elec_good.shape)

        #Filter data for one elec (all trials):
        data_elec_bad = np.array(data_elec_bad, dtype='float64')
        data_elec_good = np.array(data_elec_good, dtype='float64')
        data_bad_to_filter = np.swapaxes(data_elec_bad, 0, 1)
        data_good_to_filter = np.swapaxes(data_elec_good, 0, 1)
        filtered_data_bad = filter_data(data_bad_to_filter, sfreq=512, l_freq=None, h_freq=low_pass_filter, method='fir', phase='zero-double')
        filtered_data_good = filter_data(data_good_to_filter, sfreq=512, l_freq=None, h_freq=low_pass_filter, method='fir', phase='zero-double')
        print ('Size of filtered data bad :', filtered_data_bad.shape, 'filtered data good : ', filtered_data_good.shape,)

        #Normalize the non-averaged data (all trials)
        times = np.arange(filtered_data_bad.shape[1])
        print ('time points : ', times.shape)
        norm_filtered_data_bad = rescale(filtered_data_bad, times=times, baseline=baseline, mode=norm_mode)
        norm_filtered_data_good = rescale(filtered_data_good, times=times, baseline=baseline, mode=norm_mode)
        print ('Size norm & filtered data 0 : ', norm_filtered_data_bad.shape, norm_filtered_data_good.shape,)

        # =======================================  STATISTICS  =====================================
        # Range of the data to compute
        data_range = range(data_to_use[0], data_to_use[1])

        # Get all three learning files (Filtered data)
        data_bad = norm_filtered_data_bad[:, data_range]
        data_good = norm_filtered_data_good[:, data_range,]
        print ('-> Shape of the selected data for learn 0', data_bad.shape, 'learn 1', data_good.shape,)
        
        n_rep = 100
        T_rep = np.array([])
        p_val_rep = np.array([])
        alpha = 0.05
        for i in range(n_rep):
            #reshape data to have the exact same nb of trials (mandatory for t-tests)
            if data_bad.shape[0] > data_good.shape[0]:
                data_bad = data_bad[np.random.randint(data_bad.shape[0], size=data_good.shape[0]), :] #reshape bad_data to fit good_data shape
                print ('rand bad matrix', data_bad.shape)
            if data_bad.shape[0] < data_good.shape[0]:
                data_good = data_good[np.random.randint(data_good.shape[0], size=data_bad.shape[0]), :]
                print ('rand good matrix', data_good.shape)
            X = data_bad - data_good #the last dimension needs to be time
            T0, p_values, H0 = permutation_t_test(X, n_permutations=1000, tail=0, n_jobs=1, verbose=None)
            T_rep = np.vstack((T_rep,T0)) if np.size(T_rep) else T0
            p_val_rep = np.vstack((p_val_rep,p_values)) if np.size(p_val_rep) else p_values
            
        idx_signif_nb = []
        for s in range(n_rep):
            if p_val_rep[s,:].min() < alpha:
                idx_signif_nb.append(1)
            else:
                idx_signif_nb.append(0)
        print (idx_signif_nb)
        if sum(idx_signif_nb) >= n_rep - n_rep*alpha:
            # save all pvalues
            fname = (save_path +su +'pvalues_good_bad_' + channel [elec] +'_'+str(elec)+'_'+label[elec]+'.npy')
            np.save(fname, p_val_rep)
            
            # plot and save pvalues
            plot_name = (save_path +su +'pvalues_good_bad_' + channel [elec] +'_'+str(elec)+'_'+label[elec]+'.png')
            times_plot = 1000 * np.arange((baseline[0] - baseline[1]), data_learn_to_plot.shape[0]-baseline[1] + baseline[0],) / sf
            #Prepare the plot
            fig = plt.figure(0, figsize=(12, 7))
            ax = fig.add_subplot(111)
            fig.subplots_adjust(top=0.85)
            ax.set_xlabel('Times (ms)', fontsize=12)
            ax.set_ylabel('pvalues Good-Bad', fontsize=12)
            #Plot the data
            BorderPlot(times_plot, p_val_rep.swapaxes(0,1), kind='sem', alpha=0.2, color=['r'], 
                       linewidth=2, ncol=1, title=su+'_pvalues_'+str(channel[elec])+'_'+str(label[elec])+' elec_num: '+str(elec)+'_nrep:'+str(n_rep),)
            plt.gca()
            lines = [0] #time vector is in ms
            addPval(plt.gca(), p_values, p=0.05, x=times_plot, y=0.5, color='b', lw=3)
            addLines(plt.gca(), vLines=lines, vColor=['firebrick'], vWidth=[2], hLines=[0], hColor=['#000000'], hWidth=[2])
            plt.grid()
            plt.savefig(plot_name, dpi=300, bbox_inches='tight')
            plt.clf()
            plt.close()


-> Olfacto loaded
Channel :  b2-b1 Label :  aHC&aHC-Ent N_trials : 21/19 Bad shape :  (2560, 21) Good shape :  (2560, 19)
Setting up low-pass filter at 10 Hz
h_trans_bandwidth chosen to be 2.5 Hz
Filter length of 1352 samples (2.641 sec) selected
Setting up low-pass filter at 10 Hz
h_trans_bandwidth chosen to be 2.5 Hz
Filter length of 1352 samples (2.641 sec) selected
Size of filtered data bad : (21, 2560) filtered data good :  (19, 2560)
time points :  (2560,)
Applying baseline correction (mode: mean)
Applying baseline correction (mode: mean)
Size norm & filtered data 0 :  (21, 2560) (19, 2560)
-> Shape of the selected data for learn 0 (21, 640) learn 1 (19, 640)
rand bad matrix (19, 640)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

KeyboardInterrupt: 